In [82]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [83]:
import re
import numpy as np
import pandas as pd
from pandas import DataFrame

# Pandas
<!-- requirement: images/Data_Frame_Data_Series.png -->
<!-- requirement: small_data/fha_by_tract.csv -->
<!-- requirement: small_data/2013_Gaz_tracts_national.tsv -->

Pandas is Python's answer to R.  It's a good tool for small data analysis -- i.e. when everything fits into memory.

The basic new "noun" in pandas is the **DataFrame**.

## Nouns (objects) in Pandas


### Data Frames


Like a table, with rows and columns (e.g. as in SQL).  Except:
  - The rows can be indexed by something interesting (there is special support for labels like categorical and timeseries data).  This is especially useful when you have timeseries data with potentially missing data points.
  - Cells can store Python objects. Like in SQL, columns are type homogeneous.
  - Instead of "NULL", the name for a non-existent value is "NA".  Unlike R, Python's data frames only support NAs in columns of some data types (basically: floating point numbers and 'objects') -- but this is mostly a non-issue (because it will "up-cast" integers to float64, etc.)
  

### Data Series:

These are named columns of a DataFrame (more correctly, a DataFrame is a dictionary of Series).  The entries of the series have homogeneous type.

![Data Frame Data Series](images/Data_Frame_Data_Series.png)
[comment]: https://docs.google.com/drawings/d/1eQOWaG37cH6Uo13zfRVPWsA77F0U4WVG2y06Z3NOtAU/edit

In [84]:
# a data frame
df1 = pd.DataFrame({
    'color': ['black', 'brown', 'gray'],
    'animal': ['cat', 'dog', 'mouse'],
    'diet': ['mice', 'kibble', 'cheese']
})

df1

,animal,color,diet
0,cat,black,mice
1,dog,brown,kibble
2,mouse,gray,cheese


In [85]:
df1['animal']  # a series

0      cat
1      dog
2    mouse
Name: animal, dtype: object

In [86]:
# the same data frame
df2 = pd.DataFrame([ #a list of tuples, each tuple is a row of the dataframe
    ('cat', 'black', 'mice'),
    ('dog', 'brown', 'kibble'),
    ('mouse', 'gray', 'cheese'),
], columns=['animal', 'color', 'diet'])

np.all(df1 == df2)

True

## Verbs (operations) in Pandas

  
Pandas provides a "batteries-included" basic data analysis:
  - **Loading data:** `read_csv`, `read_table`, `read_sql`, and `read_html`
  - **Selection, filtering, and aggregation** (i.e. SQL-type operations): There's a special syntax for selecting.  There's the `merge` method for joining.  There's also an easy syntax for what in SQL is a mouthful: Creating a new column whose value is computed from another column -- with the bonus that now the computations can use the full power of Python (though it might be faster if it didn't).
  - **"Pivot table" style aggregation:** If you're an Excel cognoscenti, you may appreciate this.
  - **NA handling:** Like R's data frames, there is good support for transforming NA values with default values / averaging tricks / etc.
  - **Basic statistics:** e.g. `mean`, `median`, `max`, `min`, and the convenient `describe`.
  - **Plugging into more advanced analytics:** Okay, this isn't batteries included.  But still, it plays reasonably with `sklearn`.
  - **Visualization:** For instance `plot` and `hist`.
  
We're demonstrate some of these operations by exploring a data set of mortgage insurance issued by the Federal Housing Authority (FHA).  The data is broken down by census tract and tells us how big of a player the FHA is in each tract (how many homes etc.). Some of these operations will be further explored in the [data aggregation notebook](IW_Pandas_Data_Aggregation.ipynb).

## Loading data 


Let's load data stored on disk into a DataFrame.  Notice that this is a CSV without a header file.

In [87]:
!head small_data/fha_by_tract.csv

08,075,,1,1,100,258,258,100
28,049,0103.01,1,1,100,71,71,100
40,003,,1,1,100,215,215,100
39,113,0603.00,3,3,100,206,206,100
12,105,0124.04,2,2,100,303,303,100
12,086,9808.00,1,1,100,188,188,100
39,035,1202.00,1,1,100,19,19,100
12,103,0207.00,2,2,100,100,100,100
36,119,0030.00,1,1,100,354,354,100
39,153,,1,1,100,213,213,100


In [88]:
names = ["State_Code", "County_Code", "Census_Tract_Number", "NUM_ALL",
        "NUM_FHA", "PCT_NUM_FHA", "AMT_ALL", "AMT_FHA", "PCT_AMT_FHA"]
# Loading a CSV file, without a header (so we have to provide field names)
df = pd.read_csv('small_data/fha_by_tract.csv', names=names)
df.head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA
0,8.0,75.0,NaN,1,1,100.0,258,258,100.0
1,28.0,49.0,103.01,1,1,100.0,71,71,100.0
2,40.0,3.0,NaN,1,1,100.0,215,215,100.0
3,39.0,113.0,603.00,3,3,100.0,206,206,100.0
4,12.0,105.0,124.04,2,2,100.0,303,303,100.0


The Pandas `read_csv` method is a powerful tool for loading a wide variety of delimited text files and overcoming many typical data input problems. Because CSV files are so popular, over time many features were added and now it can handle pretty much any CSV file you are likely to find.

You can access the `read_csv` [documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) to read about all of the parameters, but here are the most useful ones.

* `sep`: Change the file delimiter. By default it is a comma, but you will find files that use tabs ('\t') and pipes. ('|')
* `header`: Row number to use for column header. Some CSV files have no header; in this case, pass None instead and use the names parameter.
* `names`: List of column names to use, overriding what `read_csv` might name the columns from reading the file.
* `index_col`: Make an index out of a column. Without this, it will create a default index of integers.
* `parse_dates`: parse certain columns as dates.
* `infer_datetime_format`: if the data in a column of dates all have the same format, this will speed up reading the file.

## Indexing and slicing data frames


Slicing a DataFrame by a column name yields a data series.

In [89]:
df['State_Code'].head()

0     8.0
1    28.0
2    40.0
3    39.0
4    12.0
Name: State_Code, dtype: float64

Slicing by a list of column names gives another data frame.

In [90]:
df[['State_Code', 'County_Code']].head()

,State_Code,County_Code
0,8.0,75.0
1,28.0,49.0
2,40.0,3.0
3,39.0,113.0
4,12.0,105.0


**Question:** What will this return?

In [91]:
type(df[['State_Code']])

pandas.core.frame.DataFrame

To slice a particular element of the frame, use the `.loc` attribute.  It takes index and (optionally) columns names.

In [92]:
df.loc[3, 'State_Code']

39.0

Both can be sliced.  Unusually for Python, both endpoints are included in the slice.

In [93]:
df.loc[0:3, 'State_Code':'Census_Tract_Number'] #inclusive-inclusive 0,1,2,3

,State_Code,County_Code,Census_Tract_Number
0,8.0,75.0,NaN
1,28.0,49.0,103.01
2,40.0,3.0,NaN
3,39.0,113.0,603.00


Position-based indexing is available in the `.iloc` attribute.

In [94]:
df.iloc[3, 0]

39.0

The usual slicing convention is used for `.iloc`.

In [147]:
df.iloc[0:3, 0:3] #pull out position(not include 3) 0,1,2

,State_Code,County_Code,Census_Tract_Number
0,8.0,75.0,NaN
1,28.0,49.0,103.01
2,40.0,3.0,NaN


**Gotcha:** Notice that position-based and index-based indexing give different results!

In [96]:
new_index = pd.RangeIndex(1, len(df) + 1) #creat a rangeindex object, index from to 1 to num+1
df.set_index(new_index).head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA
1,8.0,75.0,NaN,1,1,100.0,258,258,100.0
2,28.0,49.0,103.01,1,1,100.0,71,71,100.0
3,40.0,3.0,NaN,1,1,100.0,215,215,100.0
4,39.0,113.0,603.00,3,3,100.0,206,206,100.0
5,12.0,105.0,124.04,2,2,100.0,303,303,100.0


In [97]:
df.set_index(new_index).loc[3, :]

State_Code              40.0
County_Code              3.0
Census_Tract_Number      NaN
NUM_ALL                  1.0
NUM_FHA                  1.0
PCT_NUM_FHA            100.0
AMT_ALL                215.0
AMT_FHA                215.0
PCT_AMT_FHA            100.0
Name: 3, dtype: float64

In [98]:
df.set_index(new_index).iloc[3, :] #'i' for index, it's purely indexical

State_Code              39.0
County_Code            113.0
Census_Tract_Number    603.0
NUM_ALL                  3.0
NUM_FHA                  3.0
PCT_NUM_FHA            100.0
AMT_ALL                206.0
AMT_FHA                206.0
PCT_AMT_FHA            100.0
Name: 4, dtype: float64

By default, rows are indexed by their position.  However, any column can be made into an index:

In [99]:
df.set_index('State_Code').head(6)

,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA
State_Code,,,,,,,,
8.0,75.0,NaN,1,1,100.0,258,258,100.0
28.0,49.0,103.01,1,1,100.0,71,71,100.0
40.0,3.0,NaN,1,1,100.0,215,215,100.0
39.0,113.0,603.00,3,3,100.0,206,206,100.0
12.0,105.0,124.04,2,2,100.0,303,303,100.0
12.0,86.0,9808.00,1,1,100.0,188,188,100.0


Multiple levels of indexing is possible:

In [100]:
df.set_index(['State_Code', 'County_Code']).head(6)

Census_Tract_Number  NUM_ALL  NUM_FHA  PCT_NUM_FHA  \
State_Code County_Code                                                       
8.0        75.0                         NaN        1        1        100.0   
28.0       49.0                      103.01        1        1        100.0   
40.0       3.0                          NaN        1        1        100.0   
39.0       113.0                     603.00        3        3        100.0   
12.0       105.0                     124.04        2        2        100.0   
           86.0                     9808.00        1        1        100.0   

                        AMT_ALL  AMT_FHA  PCT_AMT_FHA  
State_Code County_Code                                 
8.0        75.0             258      258        100.0  
28.0       49.0              71       71        100.0  
40.0       3.0              215      215        100.0  
39.0       113.0            206      206        100.0  
12.0       105.0            303      303        100.0  
           86.0             188      188        100.0

An index can be turned back into a column:

In [101]:
df.set_index('State_Code').reset_index().head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA
0,8.0,75.0,NaN,1,1,100.0,258,258,100.0
1,28.0,49.0,103.01,1,1,100.0,71,71,100.0
2,40.0,3.0,NaN,1,1,100.0,215,215,100.0
3,39.0,113.0,603.00,3,3,100.0,206,206,100.0
4,12.0,105.0,124.04,2,2,100.0,303,303,100.0


## Indices in Pandas


Pandas indices allow us to use meaningful information to organize data in the table.  Elements are associated based on their index, not their order.

In [102]:
s1 = pd.Series([1,2,3], index=['a', 'b', 'c'])
s2 = pd.Series([3,2,1], index=['c', 'b', 'a'])
s1 + s2

a    2
b    4
c    6
dtype: int64

In [103]:
s3 = pd.Series([3,2,1], index=['c', 'd', 'e'])
s1 + s3

a    NaN
b    NaN
c    6.0
d    NaN
e    NaN
dtype: float64

Missing values get a `NaN`, but this can be replaced by a fill value of your choice.

In [104]:
s1.add(s3, fill_value=0)

a    1.0
b    2.0
c    6.0
d    2.0
e    1.0
dtype: float64

Notice that you can also set and reset the index.

In [105]:
df.head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA
0,8.0,75.0,NaN,1,1,100.0,258,258,100.0
1,28.0,49.0,103.01,1,1,100.0,71,71,100.0
2,40.0,3.0,NaN,1,1,100.0,215,215,100.0
3,39.0,113.0,603.00,3,3,100.0,206,206,100.0
4,12.0,105.0,124.04,2,2,100.0,303,303,100.0


In [106]:
df.set_index('State_Code').head()

,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA
State_Code,,,,,,,,
8.0,75.0,NaN,1,1,100.0,258,258,100.0
28.0,49.0,103.01,1,1,100.0,71,71,100.0
40.0,3.0,NaN,1,1,100.0,215,215,100.0
39.0,113.0,603.00,3,3,100.0,206,206,100.0
12.0,105.0,124.04,2,2,100.0,303,303,100.0


## Adding and dropping data


We can add columns to a DataFrame by slicing it with a new column name.

In [107]:
df['GEOID'] = (df['Census_Tract_Number']*100 + 10**6 * df['County_Code']
               + 10**9 * df['State_Code'])   # A computed field!
df.head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID
0,8.0,75.0,NaN,1,1,100.0,258,258,100.0,NaN
1,28.0,49.0,103.01,1,1,100.0,71,71,100.0,2.804901e+10
2,40.0,3.0,NaN,1,1,100.0,215,215,100.0,NaN
3,39.0,113.0,603.00,3,3,100.0,206,206,100.0,3.911306e+10
4,12.0,105.0,124.04,2,2,100.0,303,303,100.0,1.210501e+10


Notice that this syntax is similar to adding a new element to a dictionary.

```python
my_dict = {'a': 0, 'b': 1}
my_dict['c'] = 36
```

Recall that a DataFrame can be thought of as a dictionary of Series. When we create a new column, we are adding a new Series to that dictionary.

We can also add rows by slicing into a DataFrame at the index of the new row using `loc`.

In [108]:
df1.head()

,animal,color,diet
0,cat,black,mice
1,dog,brown,kibble
2,mouse,gray,cheese


In [109]:
df1.loc[10] = {'animal': 'horse', 'color': 'brown', 'diet': 'grass'}
df1.head()

,animal,color,diet
0,cat,black,mice
1,dog,brown,kibble
2,mouse,gray,cheese
10,horse,brown,grass


We can also remove data from a DataFrame. To drop a row:

In [110]:
df.drop(0).head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID
1,28.0,49.0,103.01,1,1,100.0,71,71,100.0,2.804901e+10
2,40.0,3.0,NaN,1,1,100.0,215,215,100.0,NaN
3,39.0,113.0,603.00,3,3,100.0,206,206,100.0,3.911306e+10
4,12.0,105.0,124.04,2,2,100.0,303,303,100.0,1.210501e+10
5,12.0,86.0,9808.00,1,1,100.0,188,188,100.0,1.208698e+10


Notice that the indices do not reset.  The index is associated with the row, not with the order.

To drop a column:

In [111]:
df.drop(df['NUM_ALL'] == 1).head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID
2,40.0,3.0,NaN,1,1,100.0,215,215,100.0,NaN
3,39.0,113.0,603.00,3,3,100.0,206,206,100.0,3.911306e+10
4,12.0,105.0,124.04,2,2,100.0,303,303,100.0,1.210501e+10
5,12.0,86.0,9808.00,1,1,100.0,188,188,100.0,1.208698e+10
6,39.0,35.0,1202.00,1,1,100.0,19,19,100.0,3.903512e+10


In [112]:
df.drop('GEOID', axis=1).head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA
0,8.0,75.0,NaN,1,1,100.0,258,258,100.0
1,28.0,49.0,103.01,1,1,100.0,71,71,100.0
2,40.0,3.0,NaN,1,1,100.0,215,215,100.0
3,39.0,113.0,603.00,3,3,100.0,206,206,100.0
4,12.0,105.0,124.04,2,2,100.0,303,303,100.0


Most operations produce copies (unless `inplace=True` is specified).  The `df` object still has the GEOID column.

In [113]:
'GEOID' in df.columns

True

## Filtering data


Now the `df[...]` notation is very flexible:
  - It accepts column names (strings and lists of strings);
  - It accepts column numbers (so long as there is no ambiguity with column names);
  - It accepts _binary data series!_
  
This means that you can write

```python
df[ (df['column_name2']==MD) & ( (df['column_name1']==5) | (df['column_name1']==6) ) ]
```

for what you would write in SQL as

```sql
SELECT * FROM df WHERE
column_name2='MD" AND (column_name1=5 OR column_name1=6)
```
            
Boolean operators on a data frame return a data series of Booleans.

In [114]:
(df['State_Code'] == 39).head()

0    False
1    False
2    False
3     True
4    False
Name: State_Code, dtype: bool

In [115]:
((df['State_Code'] == 39) & (df['Census_Tract_Number'] == 9613)).head()

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [116]:
df[(df['State_Code'] == 39) & (df['Census_Tract_Number'] == 9613)]

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID
51958,39.0,143.0,9613.0,21,2,9.52381,2110,262,12.41710,3.914396e+10
57340,39.0,31.0,9613.0,21,2,9.52381,2533,203,8.01421,3.903196e+10


**Note:** selecting rows by binary data series only if they share the same data index!

There is also a `query` method that is very similar to a SQL statement's where clause. This approach gets the same result and is easier to read.

In [117]:
df.query('State_Code == 39 and Census_Tract_Number == 9613')

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID
51958,39.0,143.0,9613.0,21,2,9.52381,2110,262,12.41710,3.914396e+10
57340,39.0,31.0,9613.0,21,2,9.52381,2533,203,8.01421,3.903196e+10


## Joining data


The analogue of a

```sql             
SELECT * 
FROM df1
INNER JOIN df2 
ON df1.field_name=df2.field_name;
```

is

```python
df_joined = df1.merge(df2, on='field_name')
```

You can also do left / right / outer joins, mix-and-match column names, etc.  For that consult the Pandas documentation. (The example below will do a left join.)

Of course, just looking at the distribution of insurance by census tract isn't interesting unless we know more about the census tract.  We'll load some data.  Notice this time, it's a TSV with headers given in the first row.

In [118]:
!head small_data/2013_Gaz_tracts_national.tsv

In [119]:
# The first row is the column names, so we don't have to specify those
df_geo = pd.read_csv('small_data/2013_Gaz_tracts_national.tsv', sep='\t')
df_geo.head()

,USPS,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,AL,1001020100,9809939,36312,3.788,0.014,32.481794,-86.490249
1,AL,1001020200,3340498,5846,1.290,0.002,32.475758,-86.472468
2,AL,1001020300,5349274,9054,2.065,0.003,32.474024,-86.459703
3,AL,1001020400,6382705,16244,2.464,0.006,32.471030,-86.444835
4,AL,1001020500,11397734,48412,4.401,0.019,32.458916,-86.421817


SyntaxError: invalid syntax (<ipython-input-145-bee987f21129>, line 1)

In [120]:
df_joined = df.merge(df_geo, on='GEOID', how='left')
df_joined.head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID,USPS,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,8.0,75.0,NaN,1,1,100.0,258,258,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28.0,49.0,103.01,1,1,100.0,71,71,100.0,2.8049e+10,MS,8162270.0,22648.0,3.151,0.009,32.365904,-90.262379
2,40.0,3.0,NaN,1,1,100.0,215,215,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39.0,113.0,603.00,3,3,100.0,206,206,100.0,3.91131e+10,OH,5382347.0,0.0,2.078,0.000,39.729932,-84.268323
4,12.0,105.0,124.04,2,2,100.0,303,303,100.0,1.2105e+10,FL,105120002.0,1800522.0,40.587,0.695,28.224489,-81.739745


## Sorting by indices and columns


We can sort by the row (or column) index.

In [121]:
df_joined.set_index('USPS').sort_index(ascending=False).head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
USPS,,,,,,,,,,,,,,,,
WY,56.0,29.0,9655.0,42,5,11.90480,7062,702,9.94053,5.603e+10,1.411144e+07,892358.0,5.448,0.345,44.521214,-109.089935
WY,56.0,1.0,9634.0,28,2,7.14286,4182,320,7.65184,5.6002e+10,1.327847e+06,0.0,0.513,0.000,41.313653,-105.588401
WY,56.0,39.0,9676.0,49,2,4.08163,41552,997,2.39940,5.604e+10,9.990324e+09,566842730.0,3857.286,218.859,44.048656,-110.585825
WY,56.0,37.0,9711.0,35,7,20.00000,5783,1091,18.86560,5.6038e+10,1.198313e+07,0.0,4.627,0.000,41.562531,-109.192861
WY,56.0,41.0,9754.0,39,8,20.51280,6013,1183,19.67400,5.6042e+10,1.321857e+08,8538723.0,51.037,3.297,41.353297,-111.007560


We can also sort by the value in a column:

In [122]:
df_joined.sort_values('AMT_FHA').head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID,USPS,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
72034,6.0,37.0,2739.02,43,0,0.0,41953,0,0.0,6.03727e+09,CA,902170.0,285874.0,0.348,0.110,33.983199,-118.466114
67153,6.0,95.0,2516.00,9,0,0.0,985,0,0.0,6.09525e+09,CA,861182.0,212332.0,0.333,0.082,38.108389,-122.262299
67154,51.0,77.0,602.01,11,0,0.0,1956,0,0.0,5.10771e+10,VA,195770944.0,2946567.0,75.588,1.138,36.661583,-81.103939
67155,39.0,35.0,1166.00,1,0,0.0,57,0,0.0,3.90351e+10,OH,878404.0,0.0,0.339,0.000,41.535167,-81.603144
67156,12.0,86.0,42.06,43,0,0.0,25559,0,0.0,1.2086e+10,FL,863259.0,281534.0,0.333,0.109,25.795657,-80.127134


## Data transformations and function application


For element-wise function application, the most straightforward thing to do is to apply NumPy functions to these objects:

In [123]:
df1 = pd.DataFrame(np.arange(24).reshape(4,6), index=list('abcd'), columns=list('ABCDEF'))
np.sin(df1)

,A,B,C,D,E,F
a,0.000000,0.841471,0.909297,0.141120,-0.756802,-0.958924
b,-0.279415,0.656987,0.989358,0.412118,-0.544021,-0.999990
c,-0.536573,0.420167,0.990607,0.650288,-0.287903,-0.961397
d,-0.750987,0.149877,0.912945,0.836656,-0.008851,-0.846220


This relies on NumPy functions automatically broadcasting themselves to work element-wise.  To apply a pure-python function to each element, use the `.applymap()` method.

In [124]:
df1.applymap(lambda x: "%.2f" % x)

,A,B,C,D,E,F
a,0.00,1.00,2.00,3.00,4.00,5.00
b,6.00,7.00,8.00,9.00,10.00,11.00
c,12.00,13.00,14.00,15.00,16.00,17.00
d,18.00,19.00,20.00,21.00,22.00,23.00


However, sometimes you want to compute things column-wise or row-wise.  In this case, you will need to use the `apply` method.  For example, the following takes the range of each column.

In [125]:
df1.apply(lambda x: x.max() - x.min())

A    18
B    18
C    18
D    18
E    18
F    18
dtype: int64

And this takes the range of reach row.

In [126]:
df1.apply(lambda x: x.max() - x.min(), axis=1)

a    5
b    5
c    5
d    5
dtype: int64

As we saw previously in the computation of the GEOID column for the FHA data, we'll often compute new columns from other columns in the DataFrame. The `apply` method lets us use custom Python functions to compute new fields.

In [127]:
df1['logA'] = df1['A'].apply(np.log1p)

## String operations and transformations


Pandas Series have many methods dedicated to processing strings. These can be useful for transforming text data, as well as filtering a data frame based on text content. These methods are [thoroughly documented with examples](https://pandas.pydata.org/pandas-docs/stable/text.html), but we'll demonstrate a few use cases.

In [148]:
df_geo[df_geo['USPS'].str.contains('^K')][['USPS', 'GEOID']].sample(5) #sample function returns randoms rows

,USPS,GEOID
26961,KS,20173006300
28179,KY,21199930800
27455,KY,21067000500
27605,KY,21093000400
27042,KS,20175966000


In [129]:
df_geo['USPS'].str.lower().unique()

array(['al', 'ak', 'az', 'ar', 'ca', 'co', 'ct', 'de', 'dc', 'fl', 'ga',
       'hi', 'id', 'il', 'in', 'ia', 'ks', 'ky', 'la', 'me', 'md', 'ma',
       'mi', 'mn', 'ms', 'mo', 'mt', 'ne', 'nv', 'nh', 'nj', 'nm', 'ny',
       'nc', 'nd', 'oh', 'ok', 'or', 'pa', 'ri', 'sc', 'sd', 'tn', 'tx',
       'ut', 'vt', 'va', 'wa', 'wv', 'wi', 'wy', 'pr'], dtype=object)

In [130]:
df_geo['USPS'].str.replace('^A', 'Z').head()

0    ZL
1    ZL
2    ZL
3    ZL
4    ZL
Name: USPS, dtype: object

## Pandas HTML data import example


Pandas takes a "batteries included" approach and throws in a whole lot of convenience functions.  For instance it has import functions for a variety of formats.  One of the pleasant surprises is a command `read_html` that's meant to automate the process of extracting tabular data from HTML.  In particular, it works pretty well with tables on Wikipedia.  

Let's do an example: We'll try to extract the list of the world's tallest structures from
http://en.wikipedia.org/wiki/List_of_tallest_buildings_and_structures_in_the_world.

In [131]:
# Fix version ID so that results are stable
url = "https://en.wikipedia.org/w/index.php?title=List_of_tallest_buildings_and_structures&oldid=783685865"

dfs = pd.read_html(url, header=0, parse_dates=False)

# There are several tables on the page.  By inspection we can figure out which one we want
tallest = dfs[3]

# The coordinates column needs to be fixed up.  This is a bit of string parsing:
def clean_lat_long(s):
    try:
        parts = s.split("/")
    except AttributeError:
        return (None, None)
    if len(parts) < 3:
        return (None, None)
    m = re.search(r"(\d+[.]\d+);[^\d]*(\d+[.]\d+)[^\d]", parts[2])
    if not m:
        return (None, None)
    return (m.group(1), m.group(2))

tallest['Clean_Coordinates'] = tallest['Coordinates'].apply(clean_lat_long)
tallest['Latitude'] = tallest['Clean_Coordinates'].apply(lambda x:x[0])
tallest['Longitude'] = tallest['Clean_Coordinates'].apply(lambda x:x[1])

# Et voila
tallest.head()

,Category,Structure,Country,City,Height (metres),Height (feet),Year built,Coordinates,Clean_Coordinates,Latitude,Longitude
0,Mixed use,Burj Khalifa,United Arab Emirates,Dubai,828.1,2717.0,2010,25°11′50.0″N 55°16′26.6″E﻿ / ﻿25.197222°N 55.2...,"(25.197222, 55.274056)",25.197222,55.274056
1,Self-supporting tower,Tokyo Skytree,Japan,Tokyo,634,2080.0,2011,35°42′36.5″N 139°48′39″E﻿ / ﻿35.710139°N 139.8...,"(35.710139, 139.81083)",35.710139,139.81083
2,Guyed steel lattice mast,KVLY-TV mast,United States,"Blanchard, North Dakota",628.8,2063.0,1963,47°20′32″N 97°17′25″W﻿ / ﻿47.34222°N 97.29028°...,"(47.34222, 97.29028)",47.34222,97.29028
3,Clock building,Abraj Al Bait Towers,Saudi Arabia,Mecca,601,1972.0,2011,21°25′08″N 39°49′35″E﻿ / ﻿21.41889°N 39.82639°...,"(21.41889, 39.82639)",21.41889,39.82639
4,Office,One World Trade Center,United States,"New York, NY",541,1776.0,2013,40°42′46.8″N 74°0′48.6″W﻿ / ﻿40.713000°N 74.01...,"(40.713000, 74.013500)",40.713000,74.013500


**Exercise**

1. Parse the table rankings of [UK universities available on Wikipedia](https://en.wikipedia.org/wiki/Rankings_of_universities_in_the_United_Kingdom)

## Pandas Timestamps


Pandas comes with excellent tools for managing temporal data. Central to this is the Timestamp class, which can infer timestamps from many sensible inputs:

In [132]:
print pd.Timestamp('July 4, 2016')
print pd.Timestamp('Monday, July 4, 2016')
print pd.Timestamp('Tuesday, July 4th, 2016')  # notice it ignored 'Tuesday'
print pd.Timestamp('Monday, July 4th, 2016 05:00 PM')

print pd.Timestamp('07/04/2016T17:20:13.123456')
print pd.Timestamp(1467651600000000000)  # number of ns since the epoch, 1/1/1970

2016-07-04 00:00:00
2016-07-04 00:00:00
2016-07-04 00:00:00
2016-07-04 17:00:00
2016-07-04 17:20:13.123456
2016-07-04 17:00:00


It can manage timestamps with timezones:

In [133]:
july4 = pd.Timestamp('Monday, July 4th, 2016 05:00 PM').tz_localize('US/Eastern')
labor_day = pd.Timestamp('9/5/2016 12:00', tz='US/Eastern')
thanksgiving = pd.Timestamp('11/24/2016 16:00')  # no timezone

Pandas can do calculations on Timestamps if they are both localized to the same timezone or neither has a timezone.

In [134]:
print labor_day - july4
# print thanksgiving - july4  # generates an error

62 days 19:00:00


The time series offsets are useful for calculating dates relative to another date. Observe it skips over weekend days but is oblivious to holidays. Pandas does support [Custom Calendars](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#holidays-holiday-calendars) if you need them.

In [135]:
from pandas.tseries.offsets import BDay, Day, BMonthEnd

print july4 + Day(5)  # 5 calendar days later, a Saturday.
print july4 + BDay(5)  # 5 business days later, or the following Monday.
print july4 - BDay(1)  # 1 business day earlier, or the previous Friday.
print july4 + BMonthEnd(1)  # last business day of the month.

2016-07-09 17:00:00-04:00
2016-07-11 17:00:00-04:00
2016-07-01 17:00:00-04:00
2016-07-29 17:00:00-04:00


Pandas can generate a range of dates. Here, we generate a list of business days in January of 2016:

In [136]:
business_days = pd.date_range('1/1/2016', '1/31/2016', freq='B')
business_days

DatetimeIndex(['2016-01-01', '2016-01-04', '2016-01-05', '2016-01-06',
               '2016-01-07', '2016-01-08', '2016-01-11', '2016-01-12',
               '2016-01-13', '2016-01-14', '2016-01-15', '2016-01-18',
               '2016-01-19', '2016-01-20', '2016-01-21', '2016-01-22',
               '2016-01-25', '2016-01-26', '2016-01-27', '2016-01-28',
               '2016-01-29'],
              dtype='datetime64[ns]', freq='B')

This can in turn be used in as a DataFrame index:

In [137]:
pd.date_range?

In [138]:
time_df = DataFrame(np.random.rand(len(business_days)),
                    index=business_days,
                    columns=['random'])
time_df.head()

,random
2016-01-01,0.777016
2016-01-04,0.102929
2016-01-05,0.798730
2016-01-06,0.752105
2016-01-07,0.717339


The same timezone functions are still available.

In [139]:
time_df.tz_localize('UTC').tz_convert('US/Pacific').head()

,random
2015-12-31 16:00:00-08:00,0.777016
2016-01-03 16:00:00-08:00,0.102929
2016-01-04 16:00:00-08:00,0.798730
2016-01-05 16:00:00-08:00,0.752105
2016-01-06 16:00:00-08:00,0.717339


We will see that a time series index can be a powerful tool for [data aggregation](IW_Pandas_Data_Aggregation.ipynb).

## DataFrames and Iterators


A data frame is an iterator that yields the column names:

In [140]:
[col for col in df]

['State_Code',
 'County_Code',
 'Census_Tract_Number',
 'NUM_ALL',
 'NUM_FHA',
 'PCT_NUM_FHA',
 'AMT_ALL',
 'AMT_FHA',
 'PCT_AMT_FHA',
 'GEOID']

Often you will need to loop through data in a DataFrame. Rather than use a `for` loop, you can convert a DataFrame into an iterator of your choosing.

The `iterrows` method creates an iterator that returns tuples of the index value and its row of data as a Pandas Series.

In [141]:
year2016 = pd.date_range(start='1/1/2016', end='12/31/2016')

daily_data_df = pd.DataFrame(np.random.rand(len(year2016)), index=year2016, columns=['random'])

In [142]:
list(daily_data_df.iterrows())[0]

(Timestamp('2016-01-01 00:00:00', freq='D'), random    0.713639
 Name: 2016-01-01 00:00:00, dtype: float64)

The `iteritems` method is similar except it iterates over columns, not rows.

In [143]:
list(daily_data_df.iteritems())[0]

('random', 2016-01-01    0.713639
 2016-01-02    0.173673
 2016-01-03    0.896073
 2016-01-04    0.345507
 2016-01-05    0.019555
 2016-01-06    0.306344
 2016-01-07    0.355899
 2016-01-08    0.518088
 2016-01-09    0.535162
 2016-01-10    0.206718
 2016-01-11    0.043380
 2016-01-12    0.959088
 2016-01-13    0.887325
 2016-01-14    0.917477
 2016-01-15    0.766248
 2016-01-16    0.503007
 2016-01-17    0.579174
 2016-01-18    0.856537
 2016-01-19    0.129920
 2016-01-20    0.557521
 2016-01-21    0.839751
 2016-01-22    0.983198
 2016-01-23    0.247908
 2016-01-24    0.564769
 2016-01-25    0.215239
 2016-01-26    0.243577
 2016-01-27    0.593332
 2016-01-28    0.499474
 2016-01-29    0.472592
 2016-01-30    0.930371
                 ...   
 2016-12-02    0.890961
 2016-12-03    0.248532
 2016-12-04    0.652425
 2016-12-05    0.703108
 2016-12-06    0.863308
 2016-12-07    0.123178
 2016-12-08    0.441710
 2016-12-09    0.003414
 2016-12-10    0.550291
 2016-12-11    0.536062
 2016-

Finally, `itertuples`. This method returns each row as a tuple. Including the index value in that tuple is optional.

In [144]:
list(daily_data_df.itertuples(index=True, name=None))[0]

(Timestamp('2016-01-01 00:00:00', freq='D'), 0.7136390414043855)

### Exercises


1. Find a CSV file somewhere on the Internet that contains data that interests you. Open the file with the Pandas `read_csv` function and plot the data.
1. Create a Pandas Timestamp for the day you were born and for today's date. How many days old are you? Seconds? Milliseconds?

### Exit Tickets


1. What is a Pandas DataFrame?
1. What is a DataFrame index? How does it differ from a regular DataFrame column?


*Copyright &copy; 2015 The Data Incubator.  All rights reserved.*